In [13]:
# add utils
import sys
sys.path.append('../../the_original_problem/')
from util.filters import filter_2d

# sample script imports
import numpy as np
import glob
from easydict import EasyDict
from matplotlib.pyplot import imread

In [15]:
def classify(im):
    '''
    Example submission for coding challenge. 
    
    Args: im (nxmx3) unsigned 8-bit color image 
    Returns: One of three strings: 'brick', 'ball', or 'cylinder'
    
    '''




    #Let's guess randomly! Maybe we'll get lucky.
    labels = ['brick', 'ball', 'cylinder']
    random_integer = np.random.randint(low = 0, high = 3)
    
    return labels[random_integer]

In [3]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [4]:
dataset_names = ['easy', 'medium_1', 'medium_2', 'hard']
classes = ['ball', 'brick', 'cylinder']
data_path = '../data' #Assuming data is stored in current directory. 
dataset_name = dataset_names[3]
image_class = classes[2]
image_paths = glob.glob(data_path + '/' + dataset_name + '/' + image_class + '/*.jpg')
image_path =  image_paths[4]
print(image_path)
im = imread(image_path)
prediction = classify(im)
if image_class == prediction:
    print('match')
    fig = figure(0, (12,12))
    imshow(im)
else:
    print('not match')

IndexError: list index out of range

ModuleNotFoundError: No module named 'filters'

In [ ]:
def canny_edge(im):
    #grayscale
    im = np.mean(im, axis = 2)
    
    #Implement Sobel kernels as numpy arrays
    Kx = np.array([[1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]])

    Ky = np.array([[1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]])
    
    Gx = filter_2d(im,Kx)
    Gy = filter_2d(im,Ky)
    #Compute Gradient Magnitude:
    gm = np.sqrt(Gx**2+Gy**2)
    
    #Arctan2 works a little better here, allowing us to avoid dividing by zero:
    gd = np.arctan2(Gy, Gx)
    
    gLocalMax = np.zeros((im.shape[0],im.shape[1]))
    gLocalMax[1:im.shape[0]-1,1:im.shape[1]-1]=gm
    
    for row in range(1,im.shape[0]-3):
        for col in range(1,im.shape[1]-3):
            v= gm[row][col]
            theta = gd[row][col]
            if (theta>=(-pi/8.0) and theta<(pi/8.0)) and (v<gm[row][col-1] or v<gm[row][col+1]):
                gLocalMax[row][col]=0
            elif (theta>=(pi/8.0) and theta<((3*pi)/8.0)) and ((v<gm[row-1][col-1] or v<gm[row+1][col+1])):
                gLocalMax[row][col]=0
            elif (theta>=((3*pi)/8.0) and theta<((5*pi)/8.0)) and (v<gm[row-1][col] or v<gm[row+1][col]):
                gLocalMax[row][col]=0
            elif (theta>=((5*pi)/8.0) and theta<((7*pi)/8.0)) and (v<gm[row+1][col-1] or v<gm[row-1][col+1]):
                gLocalMax[row][col]=0
            elif ((theta>=((7*pi)/8.0) or theta<((-7*pi)/8.0)) and ((v<gm[row][col-1] or v<gm[row][col+1]))):
                gLocalMax[row][col]=0 
            elif (theta>=((-7*pi)/8.0) and theta<((-5*pi)/8.0)) and ((v<gm[row-1][col-1] or v<gm[row+1][col+1])):
                gLocalMax[row][col]=0
            elif (theta>=((-5*pi)/8.0) and theta<((-3*pi)/8.0)) and ((v<gm[row-1][col] or v<gm[row+1][col])):
                gLocalMax[row][col]=0
            elif (theta>=((-3*pi)/8.0) and theta<((-pi)/8.0)) and ((v<gm[row+1][col-1] or v<gm[row-1][col+1])):
                gLocalMax[row][col]=0
    
    #Double threshold
    strongEdges = (gLocalMax > 30)

    #Strong has value 2, weak has value 1
    thresholdedEdges = np.array(strongEdges, dtype=np.uint8) + (gLocalMax > 20)

    #Tracing edges with hysteresis	
    #Find weak edge pixels near strong edge pixels
    finalEdges = strongEdges.copy()
    currentPixels = []
    for r in range(1, im.shape[0]-1):
        for c in range(1, im.shape[1]-1):
            if thresholdedEdges[r, c] != 1:
                continue #Not a weak pixel

            #Get 3x3 patch	
            localPatch = thresholdedEdges[r-1:r+2,c-1:c+2]
            patchMax = localPatch.max()
            if patchMax == 2:
                currentPixels.append((r, c))
                finalEdges[r, c] = 1

    #Extend strong edges based on current pixels
    while len(currentPixels) > 0:
        newPix = []
        for r, c in currentPixels:
            for dr in range(-1, 2):
                for dc in range(-1, 2):
                    if dr == 0 and dc == 0: continue
                    r2 = r+dr
                    c2 = c+dc
                    if thresholdedEdges[r2, c2] == 1 and finalEdges[r2, c2] == 0:
                        #Copy this weak pixel to final result
                        newPix.append((r2, c2))
                        finalEdges[r2, c2] = 1
        currentPixels = newPix
 
    return finalEdges

In [ ]:
G = canny_edge(im)
print(G.shape)
fig = figure(0, (12,12))
imshow(G*255)
colorbar()

In [ ]:
for row in range(G.shape[0]):
    for col in range(G.shape[1]):
        if G[row][col]>0:
            print(G[row][col])

In [ ]:
G[254][254]

In [ ]:
G = canny_edge(im)
print(G.shape)
fig = figure(0, (12,12))
imshow(G)

In [ ]:
for i in range(1,im.shape[1]-2):
    print(i)